In [0]:
from google.colab import files

uploaded = files.upload()

Saving train_comment_small_50.csv to train_comment_small_50 (1).csv


In [0]:
import pandas as pd

df = pd.read_csv('train_comment_small_50.csv', sep=',')

In [0]:
import re

def clean_comment(text):
    # Strip HTML tags
    text = re.sub('<[^<]+?>', ' ', text)
 
    # Strip escaped quotes
    text = text.replace('\\"', '')
 
    # Strip quotes
    text = text.replace('"', '')
 
    return text

In [0]:
df["comment_text"]

0     "\n \n I'm sure if you worked on articles othe...
1     D'aww! He matches this background colour I'm s...
2     Hey man, I'm really not trying to edit war. It...
3     "\n More\n I can't make any real suggestions o...
4      REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski
5     The Mitsurugi point made no sense - why not ar...
6     Don't mean to bother you \n \n I see that you'...
7     "\n \n  Regarding your recent edits \n \n Once...
8     "\n Good to know. About me, yeah, I'm studying...
9     "\n \n  Snowflakes are NOT always symmetrical!...
10    "\n \n  The Signpost: 24 September 2012 \n \n ...
11    "\n \n Re-considering 1st paragraph edit?\n I ...
12    Bye! \n \n Don't look, come or think of commin...
13    How could I post before the block expires? The...
14    Not sure about a heading of 'Fight for Freedom...
15    Praise \n \n looked at this article about 6 mo...
16    I was able to post the above list so quickly b...
17    "\n Well, not ""before the process"" but "

In [0]:
from sklearn.model_selection import train_test_split

df['cleaned_comment'] = df['comment_text'].apply(clean_comment)
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_comment'], df['toxic'], test_size=0.2)

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
 
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), 
                             lowercase=True, min_df=3, max_df=0.9, max_features=5000)
X_train_onehot = vectorizer.fit_transform(X_train)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
 
nn = Sequential()
 
nn.add(Dense(units=500, activation='relu', input_dim=len(vectorizer.get_feature_names())))
nn.add(Dense(units=1, activation='sigmoid'))
 
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               28500     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 501       
Total params: 29,001
Trainable params: 29,001
Non-trainable params: 0
_________________________________________________________________


In [0]:
nn.fit(X_train_onehot[:-20], y_train[:-20], 
          epochs=5, batch_size=128, verbose=1, 
          validation_data=(X_train_onehot[-20:], y_train[-20:]))

Train on 20 samples, validate on 20 samples
Epoch 1/5
20/20 [==============================] - 1s 33ms/step - loss: 0.6924 - acc: 0.6000 - val_loss: 0.6625 - val_acc: 0.7500
Epoch 2/5
20/20 [==============================] - 0s 522us/step - loss: 0.6551 - acc: 0.7500 - val_loss: 0.6283 - val_acc: 0.9000
Epoch 3/5
20/20 [==============================] - 0s 179us/step - loss: 0.6162 - acc: 0.8000 - val_loss: 0.5948 - val_acc: 0.9500
Epoch 4/5
20/20 [==============================] - 0s 269us/step - loss: 0.5791 - acc: 0.8500 - val_loss: 0.5630 - val_acc: 0.9500
Epoch 5/5
20/20 [==============================] - 0s 250us/step - loss: 0.5450 - acc: 0.8500 - val_loss: 0.5331 - val_acc: 0.9500


In [0]:
scores = nn.evaluate(vectorizer.transform(X_test), y_test, verbose=1)
print("Accuracy:", scores[1]) 

10/10 [==============================] - 0s 135us/step
Accuracy: 0.8999999761581421


In [0]:
nn.save('nn.hd5')